In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pdfplumber
import re

In [3]:
with pdfplumber.open("Invoice.pdf") as pdf_file:
    data = []    
    for i in range(0, len(pdf_file.pages)):
        page_data = pdf_file.pages[i]
        data.append(page_data.extract_text())

In [4]:
Incidentals = []
for i in range(0,len(data)):
    text_list = data[i].split('\n') # split lines using \n escape char
    Ientries = [e for e in text_list if "Incid" in e]
    Incidentals = Incidentals + Ientries

# Pop the clauses from the list of incidentals
Clauses = [e for e in Incidentals if not any(c.isdigit() for c in e)]
Fees = [e for e in Incidentals if any(c.isdigit() for c in e)]

In [5]:
df = None
for i in range(0,len(Fees)):
    '''Create categories and text field fillings
    Copnvert the text values from fees list into dict
    The dictionary will be used to make a pandas df
    '''

    # Create some categories
    fee_cats = Fees[i].split(' ')[1].split(':')

    # Extract Actual Fee itself
    # fee = re.findall("\d+\.\d+", Fees[i])
    fee = float(Fees[i].split('$')[1].strip(' '))

    # Extract fee description
    desc = Fees[i].strip('Incid. ')
    for cat_num in range(0,len(fee_cats)):
        desc = desc.strip(fee_cats[cat_num])
    desc = desc.split('$')[0].strip(' ').strip(':') # 1st (0) entry is description


    '''Now we can construct the dictionary'''
    my_dict = {}
    
    # Extract category fields
    for cat_num in range(0,len(fee_cats)):
        my_dict["Category " + str(cat_num)] = fee_cats[cat_num]
    
    my_dict["Fees"] = fee
    my_dict["Description"] = desc

    if df is None: # Check to see if df has been created
        df = pd.DataFrame([my_dict])

    elif df is not None:
        '''
        Since this is the case in which the dataframe has been created already
        we can simply append the data to the existing dataframe'''
        df = df.append(my_dict, ignore_index=True)
    

In [ ]:
sns.set(style='whitegrid', context='talk')
ax = sns.barplot(data=df.Fees)
# Set up the matplotlib figure


In [15]:
def generate_breakdown_df(df,category='Category 1'):
    df = df[[category,'Fees']]
    cats = df[category].unique()
    breakdown_df = None
    my_dict = {}

    for cat in cats:
        fee_total = df.loc[df[category] == cat, 'Fees'].sum()
        fee_total = round(fee_total,2)
        # print('You pay $' + str(fee_total) + ' towards ' + str(cat))

        my_dict[category] = cat
        my_dict['Fees'] = fee_total

        if breakdown_df is None: # Check to see if df has been created
            breakdown_df = pd.DataFrame([my_dict])
        elif breakdown_df is not None:
            breakdown_df = df.append(my_dict, ignore_index=True)
    
    return breakdown_df

In [16]:
x = generate_breakdown_df(df=df)
x

,Category 1,Fees
0,KPE,138.80
1,UTSG,84.34
2,UTSG,81.47
3,UTSG,89.89
4,Eng.,4.50
...,...,...
89,UTSU,2.77
90,UTSU,0.50
91,UTSU,0.76
92,Varsity,2.93


In [76]:
print(df.Fees.sum())

1634.8


In [ ]:
 # Create some categories
fee_cats = Fees[i].split(' ')[i].split(':')

# Extract Actual Fee itself
fee = re.findall("\d+\.\d+", Fees[i])\

# Extract fee description
desc = Fees[0].strip('Incid. ')
for i in range(0,len(fee_cats)):
    desc = desc.strip(fee_cats[i])
desc = desc.split('$')[0].strip(' ').strip(':')


''' Now we can construct the dictionary'''
my_dict = {}
    
# Extract category fields
for cat_num in range(0,len(fee_cats)):
    my_dict["Category " + str(cat_num)] = fee_cats[cat_num]
    
my_dict["Fees"] = fee
my_dict["Description"] = desc

pd.DataFrame.from_dict(my_dict)